# Model used
I'll be using llama.cpp so I need a model in GGUF format,
huggingface has lots of models and you can filter with GGUF tag

the model I will be using is 
meta-llama-3-8b-instruct.Q4_K_S.gguf which is a quantized model of llama-3-7b 
from https://huggingface.co/SanctumAI/Meta-Llama-3-8B-Instruct-GGUF 

Uncomment the line below to download it, or use huggingface hub through python api

In [1]:
# mkdir models
# !curl -o ./models https://huggingface.co/SanctumAI/Meta-Llama-3-8B-Instruct-GGUF/blob/main/meta-llama-3-8b-instruct.Q4_K_M.gguf 

In [2]:
# llm = Llama.from_pretrained(
#     repo_id="SanctumAI/Meta-Llama-3-8B-Instruct-GGUF",
#     filename="*Q4_k_M.gguf",
#     verbose=True
# )

## Imports

In [3]:
from llama_cpp import Llama
import textwrap
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import torch
import gc
import transformers
import os
from dotenv import load_dotenv

load_dotenv()


/home/userzero/.local/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-22 09:52:47.092799: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 09:52:47.120500: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 09:52:47.120521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been

True

# LLM using llama.cpp without RAG

In [4]:
llm = Llama(
    model_path="./models/meta-llama-3-8b-instruct.Q4_K_M.gguf",
    n_gpu_layers=-1,
    n_ctx=4096,
    flash_attn=True

)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./models/meta-llama-3-8b-instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.a

In [20]:
output = llm(
    # "Who was Zed main rival in Runeterra?",
    """remove unneeded formatting and links but don't change pure text format:
    Title: Kayle: {{Game
banner|Kayle|background=06MT008-full.png|offset=top:-45%;left:0%;}}
{{Champion bio|Kayle |render = <gallery> Kayle Zealous Render.png|Main
(Zealous) Kayle Aflame Render.png|Arisen Kayle Arisen
Render.png|Aflame Kayle Transcendent Render.png|Transcendent
</gallery> |real_name = |nickname = *
{{cbi|Morgana|Sister|nolink=true}} * {{cbi|Mihira|Dear
One|nolink=true|artwork=true}} |alias = * The Righteous * The
Protector * Bringer of Justice * The Winged Protector * Winged
Protector of Demacia |species = [[Human]] <small>([[Magical
Alteration|Magically Altered]]; [[Aspect Host]] Candidate)</small>
|pronoun = She/Her |origin = {{age}} |weapon = * {{ai|Divine
Ascent|Kayle|Ascended Wings}} * {{ai|Divine Judgment|Kayle|Celestial
Fire Magic}} * {{ai|Starfire Spellblade|Kayle|Virtue (Named Sword)}}
|status = Alive |originplace = [[Mount Targon]] <small>(Ancient
Times)</small> |residence = [[Mount Targon]] <small>(No Fixed
Abode)</small> |family = * {{cbi|Mihira}} <small>(Mother)</small> *
[[Kilam]] {{dead}} <small>(Father)</small> * {{cbi|Morgana}}
<small>(Twin Sister)</small> |pet = |occupation = * Protector of
Justice * Guardian |region = * {{fi|Demacia}} * {{fi|Mount Targon}}
|faction = |related = Morgana, Mihira, Aatrox, Ryze, Xolaani }}
[[File:Kayle_Morgana_Divider_01.png|center|400px]] Born to a Targonian
Aspect at the height of the Rune Wars, {{cbi|Kayle}} honored her
mother's legacy by fighting for justice on wings of divine flame. She
and her twin sister {{cbi|Morgana}} were the protectors of
{{fi|Demacia}} for many years—until '''Kayle''' became disillusioned
with the repeated failings of mortals, and abandoned this realm
altogether. Still, legends are told of her punishing the unjust with
her fiery swords, and many hope that she will one day return...
{{Background quote|No human is perfect. But I am not a human.|Kayle}}
==Background== ===Early life=== Kayle and her twin sister
{{cbi|Morgana}} were born on Mount Targon during the height of the
Rune Wars to [[Minor_Characters#Kilam|Kilam]] and {{cbi|Mihira}}, the
current Aspect of Justice. Due to Mihira ascending while pregnant, the
sisters were born with a fraction of her celestial power. As the war
raged on, Mihira's celestial power began to overshadow her personality
and affections as she dedicated more and more of her time to her
duties as an aspect. Fearing what Mihira was becoming, as well as the
enemies drawn to her light, Kilam resolved to take their daughters to
safety. He fled Targon with his children and they would sail to the
land that would be known as [[Demacia]]. ===Becoming Demacia's Winged
Protector=== The sisters grew to having distinct and clashing
personalities.
""",
      max_tokens=None, # set to None to generate up to the end of the context window
      stop=["Q:"], # Stop generating just before the model would generate a new question
      echo=False,
      temperature=0.75
) # Generate a completion, can also call create_completion
# print(output)
print(textwrap.fill(output["choices"][0]["text"], width=70))

Llama.generate: prefix-match hit

llama_print_timings:        load time =     349.62 ms
llama_print_timings:      sample time =     201.97 ms /   300 runs   (    0.67 ms per token,  1485.38 tokens per second)
llama_print_timings: prompt eval time =     540.09 ms /   803 tokens (    0.67 ms per token,  1486.80 tokens per second)
llama_print_timings:        eval time =    5263.97 ms /   299 runs   (   17.61 ms per token,    56.80 tokens per second)
llama_print_timings:       total time =    6220.67 ms /  1102 tokens


    remove unneeded formatting and links but don't change pure text
format : Title: Kayle: Kayle, born to a Targonian Aspect at the height
of the Rune Wars, honored her mother's legacy by fighting for justice
on wings of divine flame. She and her twin sister Morgana were the
protectors of Demacia for many years—until Kayle became disillusioned
with the repeated failings of mortals, and abandoned this realm
altogether. Still, legends are told of her punishing the unjust with
her fiery swords, and many hope that she will one day return... No
human is perfect. But I am not a human. Background: Early life: Kayle
and her twin sister Morgana were born on Mount Targon during the
height of the Rune Wars to Kilam and Mihira, the current Aspect of
Justice. Due to Mihira ascending while pregnant, the sisters were born
with a fraction of her celestial power. As the war raged on, Mihira's
celestial power began to overshadow her personality and affections as
she dedicated more and more of her time t

Born to a Targonian
Aspect at the height of the Rune Wars, {{cbi|Kayle}} honored her
mother's legacy by fighting for justice on wings of divine flame. She
and her twin sister {{cbi|Morgana}} were the protectors of
{{fi|Demacia}} for many years—until '''Kayle''' became disillusioned
with the repeated failings of mortals, and abandoned this realm
altogether. Still, legends are told of her punishing the unjust with
her fiery swords, and many hope that she will one day return...
{{Background quote|No human is perfect. But I am not a human.|Kayle}}
==Background== ===Early life=== Kayle and her twin sister
{{cbi|Morgana}} were born on Mount Targon during the height of the
Rune Wars to [[Minor_Characters#Kilam|Kilam]] and {{cbi|Mihira}}, the
current Aspect of Justice. Due to Mihira ascending while pregnant, the
sisters were born with a fraction of her celestial power. As the war
raged on, Mihira's celestial power began to overshadow her personality
and affections as she dedicated more and more of her time to her
duties as an aspect. Fearing what Mihira was becoming, as well as the
enemies drawn to her light, Kilam resolved to take their daughters to
safety. He fled Targon with his children and they would sail to the
land that would be known as [[Demacia]]. ===Becoming Demacia's Winged
Protector=== The sisters grew to having distinct and clashing
personalities.

In [31]:
import re

# Input text
text = """
Title: Kayle: {{Game
banner|Kayle|background=06MT008-full.png|offset=top:-45%;left:0%;}}
{{Champion bio|Kayle |render = <gallery> Kayle Zealous Render.png|Main
(Zealous) Kayle Aflame Render.png|Arisen Kayle Arisen
Render.png|Aflame Kayle Transcendent Render.png|Transcendent
</gallery> |real_name = |nickname = *
{{cbi|Morgana|Sister|nolink=true}} * {{cbi|Mihira|Dear
One|nolink=true|artwork=true}} |alias = * The Righteous * The
Protector * Bringer of Justice * The Winged Protector * Winged
Protector of Demacia |species = [[Human]] <small>([[Magical
Alteration|Magically Altered]]; [[Aspect Host]] Candidate)</small>
|pronoun = She/Her |origin = {{age}} |weapon = * {{ai|Divine
Ascent|Kayle|Ascended Wings}} * {{ai|Divine Judgment|Kayle|Celestial
Fire Magic}} * {{ai|Starfire Spellblade|Kayle|Virtue (Named Sword)}}
|status = Alive |originplace = [[Mount Targon]] <small>(Ancient
Times)</small> |residence = [[Mount Targon]] <small>(No Fixed
Abode)</small> |family = * {{cbi|Mihira}} <small>(Mother)</small> *
[[Kilam]] {{dead}} <small>(Father)</small> * {{cbi|Morgana}}
<small>(Twin Sister)</small> |pet = |occupation = * Protector of
Justice * Guardian |region = * {{fi|Demacia}} * {{fi|Mount Targon}}
|faction = |related = Morgana, Mihira, Aatrox, Ryze, Xolaani }}
[[File:Kayle_Morgana_Divider_01.png|center|400px]] Born to a Targonian
Aspect at the height of the Rune Wars, {{cbi|Kayle}} honored her
mother's legacy by fighting for justice on wings of divine flame. She
and her twin sister {{cbi|Morgana}} were the protectors of
{{fi|Demacia}} for many years—until '''Kayle''' became disillusioned
with the repeated failings of mortals, and abandoned this realm
altogether. Still, legends are told of her punishing the unjust with
her fiery swords, and many hope that she will one day return...
{{Background quote|No human is perfect. But I am not a human.|Kayle}}
==Background== ===Early life=== Kayle and her twin sister
{{cbi|Morgana}} were born on Mount Targon during the height of the
Rune Wars to [[Minor_Characters#Kilam|Kilam]] and {{cbi|Mihira}}, the
current Aspect of Justice. Due to Mihira ascending while pregnant, the
sisters were born with a fraction of her celestial power. As the war
raged on, Mihira's celestial power began to overshadow her personality
and affections as she dedicated more and more of her time to her
duties as an aspect. Fearing what Mihira was becoming, as well as the
enemies drawn to her light, Kilam resolved to take their daughters to
safety. He fled Targon with his children and they would sail to the
land that would be known as [[Demacia]]. ===Becoming Demacia's Winged
Protector=== The sisters grew to having distinct and clashing
personalities.
"""

# # Regex to remove unneeded formatting and links
# cleaned_text = re.sub(r'\{\{[^{}]*\}\}|\[\[[^\]]*\]\]|\<[^>]*\>|==[^=]*==|\*|\|', '', text)

# # Remove extra whitespaces
# cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text).strip()

# print(cleaned_text)

def clean_text(text):
    # Remove {{}} formatting
    text = re.sub(r'\{\{[^}]*\}\}', '', text)
    
    # Remove [[]] links
    text = re.sub(r'\[\[[^]]*\]\]', '', text)
    
    # Remove <> tags
    text = re.sub(r'<[^>]*>', '', text)
    
    # Remove single brackets []
    text = re.sub(r'\[[^]]*\]', '', text)
    
    # Remove asterisks
    text = re.sub(r'\*', '', text)
    
    # Remove multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    
    # Remove equal signs
    pattern = r'\=+([^\=]*?)\=+'
    # Replace the matched pattern
    text = re.sub(pattern, r'\1:', text)

    # Trim leading and trailing whitespace
    text = text.strip()
    
    return text
print(clean_text(text=text))


Title: Kayle: |alias  The Righteous The Protector Bringer of Justice The Winged Protector Winged Protector of Demacia |species : (; Candidate) |pronoun  She/Her |origin : |weapon  |status : Alive |originplace  (Ancient Times) |residence : (No Fixed Abode) |family  (Mother) (Father) (Twin Sister) |pet : |occupation  Protector of Justice Guardian |region : |faction  |related : Morgana, Mihira, Aatrox, Ryze, Xolaani }} Born to a Targonian Aspect at the height of the Rune Wars, honored her mother's legacy by fighting for justice on wings of divine flame. She and her twin sister were the protectors of for many years—until '''Kayle''' became disillusioned with the repeated failings of mortals, and abandoned this realm altogether. Still, legends are told of her punishing the unjust with her fiery swords, and many hope that she will one day return... Background: Early life: Kayle and her twin sister were born on Mount Targon during the height of the Rune Wars to and , the current Aspect of Jus

In [24]:
print(text)


Title: Kayle: {{Game
banner|Kayle|background=06MT008-full.png|offset=top:-45%;left:0%;}}
{{Champion bio|Kayle |render = <gallery> Kayle Zealous Render.png|Main
(Zealous) Kayle Aflame Render.png|Arisen Kayle Arisen
Render.png|Aflame Kayle Transcendent Render.png|Transcendent
</gallery> |real_name = |nickname = *
{{cbi|Morgana|Sister|nolink=true}} * {{cbi|Mihira|Dear
One|nolink=true|artwork=true}} |alias = * The Righteous * The
Protector * Bringer of Justice * The Winged Protector * Winged
Protector of Demacia |species = [[Human]] <small>([[Magical
Alteration|Magically Altered]]; [[Aspect Host]] Candidate)</small>
|pronoun = She/Her |origin = {{age}} |weapon = * {{ai|Divine
Ascent|Kayle|Ascended Wings}} * {{ai|Divine Judgment|Kayle|Celestial
Fire Magic}} * {{ai|Starfire Spellblade|Kayle|Virtue (Named Sword)}}
|status = Alive |originplace = [[Mount Targon]] <small>(Ancient
Times)</small> |residence = [[Mount Targon]] <small>(No Fixed
Abode)</small> |family = * {{cbi|Mihira}} <small>(Moth

In [9]:

print(textwrap.fill(output["choices"][0]["text"], width=70))

Kayle was born with a natural affinity for justice, her early years
were spent studying and training in combat. She also learned how to
harness the power of her divine wings and channel it into powerful
spells. In contrast, Morgana's connection to Targonian magic was more
focused on illusions and deception. As they grew older, their
differing personalities made them natural rivals, with Kayle seeing
herself as a warrior and protector, while Morgana saw herself as an
artist and manipulator.


In [69]:
del llm
_ = gc.collect()

# Put everything together - RAG

In [5]:
df = pd.read_csv("./data/chunks_data.csv")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch_embeddings = torch.load("./data/embeddings_torch.pt", map_location="cpu")
tokenizer = transformers.AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", token=os.environ["HF_TOKEN_KEY"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
#TODO: move embedding model to CPU
# model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True, model_kwargs={"torch_dtype":torch.bfloat16})
model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True, device="cpu")

In [12]:
question = "Who was Zed main rival in Runeterra?"
question = "kayle bio"

text_embeddings = model.encode(question, batch_size=1, show_progress_bar=False)
# text_embeddings_torch = torch.from_numpy(text_embeddings).to(device)

cos_sim_score = cos_sim(a=text_embeddings, b=torch_embeddings)[0]

top_result = torch.topk(cos_sim_score, k=5)
top_result

torch.return_types.topk(
values=tensor([0.5684, 0.5569, 0.5514, 0.5510, 0.5479]),
indices=tensor([ 419, 3609, 3611,   16, 5768]))

In [10]:
#TODO: stopped here
context = ""
for i in top_result[1]:
    context = context + " \n" + "Title: " + df.iloc[int(i)]["title"] + ": "+ df.iloc[int(i)]["text"]
    context_encoded = tokenizer.encode(context)
    if len(context_encoded) > 5700:
        break

In [11]:
print(textwrap.fill(context, width=70))

  Title: Kayle: {{Game
banner|Kayle|background=06MT008-full.png|offset=top:-45%;left:0%;}}
{{Champion bio|Kayle |render = <gallery> Kayle Zealous Render.png|Main
(Zealous) Kayle Aflame Render.png|Arisen Kayle Arisen
Render.png|Aflame Kayle Transcendent Render.png|Transcendent
</gallery> |real_name = |nickname = *
{{cbi|Morgana|Sister|nolink=true}} * {{cbi|Mihira|Dear
One|nolink=true|artwork=true}} |alias = * The Righteous * The
Protector * Bringer of Justice * The Winged Protector * Winged
Protector of Demacia |species = [[Human]] <small>([[Magical
Alteration|Magically Altered]]; [[Aspect Host]] Candidate)</small>
|pronoun = She/Her |origin = {{age}} |weapon = * {{ai|Divine
Ascent|Kayle|Ascended Wings}} * {{ai|Divine Judgment|Kayle|Celestial
Fire Magic}} * {{ai|Starfire Spellblade|Kayle|Virtue (Named Sword)}}
|status = Alive |originplace = [[Mount Targon]] <small>(Ancient
Times)</small> |residence = [[Mount Targon]] <small>(No Fixed
Abode)</small> |family = * {{cbi|Mihira}} <small>(Mot

In [23]:

context_encoded = tokenizer.encode(context)

/home/userzero/open/huggingface/transformers/src/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
len(context_encoded)

23348

In [ ]:
del model
torch.cuda.empty_cache()
_ = gc.collect()

In [ ]:
prompt = f"""
You are a wise old Monk that knows all the stories about League of legends, have knowledge of the lore and diffenrly better than necrit.
With the following context: {context} from League of Legend, Answer this question:{question} in a very articulate way 
"""


In [ ]:

prompt = f"""
You are a wise old Monk that knows all the stories about League of legends, have knowledge of the lore and diffenrly better than necrit.
With the following context: {context} from League of Legend, Answer this question:{question} in a very articulate way 
"""
output = llm(
    prompt,
      max_tokens=1024, # set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=True,
      temperature=0.75
) # Generate a completion, can also call create_completion
print(output)

In [68]:
del llm
_ = gc.collect()